In [15]:
import pandas as pd
import string
from nltk.corpus import stopwords
import spacy
import os
import pickle

In [27]:
# Load spaCy Hebrew model (or multilingual model for better tokenization)
nlp = spacy.load("xx_ent_wiki_sm")

# Function to preprocess Hebrew text
def preprocess_hebrew(text):
    import re

    # Remove backslash-prefixed tags (e.g., \n, \t)
    text = re.sub(r"\\[a-zA-Z]+", "", text)

    # Tokenization (using spaCy's tokenizer)
    doc = nlp(text)

    # List to store cleaned tokens
    tokens = []
    
    for token in doc:
        # Keep tokens like "headline:" and "text:" with the colon
        if token.text in ["headline:", "text:"]:
            tokens.append(token.text)
        # Keep Hebrew words, non-backslash English words, and exclude punctuation and stop words
        elif not token.is_stop and not token.is_punct:
            tokens.append(token.text)

    # Rejoin tokens into cleaned text
    cleaned_text = ' '.join(tokens)
    return cleaned_text


In [29]:
# Load data
DATA_PATH = '/sise/home/omertole/chatdbg/data/'
pickle_file = os.path.join(DATA_PATH, 'first_data_batch.pkl')
df = pd.read_pickle(pickle_file)

# Apply preprocessing to Hebrew text
df['clean_combined'] = df['combined'].apply(preprocess_hebrew)

In [30]:
df['clean_combined' and df['clean_combined'] != ''].head()

,book_id,type,unit,headline,from_date,to_date,additional_info,additional_info_2,combined_info,combined,clean_combined
0,241083,ארכיון בן גוריון - מסמך,נאומים ומאמרים,"לעם ישראל, נאום לרגל ראש-השנה, מלחמה, נצחון, כ...",24/09/1948,24/09/1948,,,,"Headline: לעם ישראל, נאום לרגל ראש-השנה, מלחמה...",Headline לעם ישראל נאום לרגל ראש השנה מלחמה נצ...
1,241089,ארכיון בן גוריון - מסמך,נאומים ומאמרים,"אדוני ראש הוועד, מוריי ורבותיי, נאום בפני וועד...",03/01/1949,03/01/1949,,,,"Headline: אדוני ראש הוועד, מוריי ורבותיי, נאום...",Headline אדוני ראש הוועד מוריי ורבותיי נאום בפ...
2,241095,ארכיון בן גוריון - מסמך,נאומים ומאמרים,"נאום לרגל יום העצמאות, חגים לאומיים, משה רבנו,...",03/05/1949,03/05/1949,,,,"Headline: נאום לרגל יום העצמאות, חגים לאומיים,...",Headline נאום לרגל יום העצמאות חגים לאומיים מש...
3,241097,ארכיון בן גוריון - מסמך,נאומים ומאמרים,"לעתידה של הקרן הקיימת לישראל, קרקעות לאום, חכי...",15/05/1948,31/12/1950,,,,"Headline: לעתידה של הקרן הקיימת לישראל, קרקעות...",Headline לעתידה של הקרן הקיימת לישראל קרקעות ל...
4,241105,ארכיון בן גוריון - מסמך,נאומים ומאמרים,"מדינת ישראל והתנועה הציונית, פתיחת הוועד הפועל...",05/05/1949,05/05/1949,,,,"Headline: מדינת ישראל והתנועה הציונית, פתיחת ה...",Headline מדינת ישראל והתנועה הציונית פתיחת הוו...


In [31]:
new_data_path = os.path.join(DATA_PATH, "prepd_data.pkl")
with open(new_data_path, "wb") as file: 
    pickle.dump(df, file)